# Surface Mapping Using AprilTags and ArUco markers

## Introduction to Fiducial Markers

Fiducial markers (for a review see[<sup>1</sup>](https://doi.org/10.1007/s10846-020-01307-9)) are specially designed visual markers used to establish spatial correspondence between different coordinate systems. They serve as reference points in images that can be reliably and accurately detected, allowing us to map between video/image coordinates and real-world coordinates. In eye-tracking applications, fiducial markers enable us to transform gaze coordinates from the camera's perspective to the coordinates of the observed surface (e.g., a computer screen).

For surface mapping, PyNeon supports the use of two widely adopted fiducial marker systems: AprilTags and ArUco markers. Both have a barcode-like appearance and encode unique IDs in their patterns.

- [AprilTag](https://april.eecs.umich.edu/software/apriltag): see references<sup>[2](https://doi.org/10.1109/ICRA.2011.5979561),[3](https://doi.org/10.1109/IROS.2016.7759617)</sup>. Pupil Labs offers AprilTag-based surface mapping in [Neon Player](https://docs.pupil-labs.com/neon/neon-player/surface-tracker/) and in [Pupil Cloud](https://docs.pupil-labs.com/neon/pupil-cloud/enrichments/marker-mapper/). However, PyNeon's implementation allows for more customizable detection parameters.

- [ArUco](https://www.uco.es/investiga/grupos/ava/portfolio/aruco/): see reference<sup>[4](https://doi.org/10.1016/j.patcog.2014.01.005)</sup>. Pre-defined ArUco dictionaries for generating and detecting these makers are integrated into OpenCV. Therefore, PyNeon uses OpenCV's ArUco module ([cv2.aruco](https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html)) for marker detection (including AprilTag).

<p><a href="https://commons.wikimedia.org/wiki/File:Comparison_of_augmented_reality_fiducial_markers.svg#/media/File:Comparison_of_augmented_reality_fiducial_markers.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/Comparison_of_augmented_reality_fiducial_markers.svg" alt="Comparison of augmented reality fiducial markers.svg" width="600"/></a>

## Reading Sample Data

To illustrate surface mapping using fiducial markers, we will use a sample dataset called "markers". the dataset includes two recordings: one with AprilTag markers and another with ArUco markers. In both recordings, a pilot participant viewed a set of artworks displayed on a computer screen. Our goal is the map the gaze and fixation data onto the computer screen using the fiducial markers present in the scene camera video.

In [ ]:
from pyneon import Dataset, get_sample_data
import matplotlib.pyplot as plt

# Load a sample recording
dataset_dir = get_sample_data("markers", format="cloud")
dataset = Dataset(dataset_dir)

Let's start with the AprilTag recording.

In [ ]:
rec = dataset.recordings[0]
print(rec)

In [ ]:
video = rec.scene_video

fig, axs = plt.subplots(1, 3, figsize=(12, 3))
video.plot_frame(380, ax=axs[0], show=False)
video.plot_frame(500, ax=axs[1], show=False)
video.plot_frame(520, ax=axs[2], show=False)
axs[0].set_title("Fixation")
axs[1].set_title("Image")
axs[2].set_title("ISI")
plt.show()

In all of the frames shown above, we can see QR-code like markers at the borders of the screen. These are called apriltags and can be used as fiducial markers to relate video to real-world coordinates. PyNeon wraps a function that performs the detection of these. For computational efficiency, we only perform one detection every 5 frames.

In [ ]:
from pathlib import Path
from pyneon import Stream
if Path("detected_markers.csv").exists():
    detected_markers = Stream("detected_markers.csv")
else:
    detected_markers = video.detect_markers("36h11")


In [ ]:
detected_markers.data.to_csv("detected_markers.csv")

In [ ]:
# Plot a few frames with detected markers
fig, axs = plt.subplots(2, 2, figsize=(10, 6), tight_layout=True)
axs = axs.flatten()
video.plot_detected_markers(detected_markers, frame_index=500, ax=axs[0], show=False)
video.plot_detected_markers(detected_markers, frame_index=1000, ax=axs[1], show=False)
video.plot_detected_markers(detected_markers, frame_index=1500, ax=axs[2], show=False)
video.plot_detected_markers(detected_markers, frame_index=2000, ax=axs[3], show=False)
plt.show()

Having detected the markers (apriltags), we now need to provide information on the real-world coordinates of our markers. This is solved via a marker_info dataframe, which we generate below.

In [ ]:
from pyneon.vis import plot_marker_layout
import pandas as pd

marker_layout = pd.DataFrame(
    {
        "marker name": [f"36h11_{i}" for i in range(6)],
        "size": 200,
        "center x": [150, 1770, 1770, 1770, 150, 150],
        "center y": [150, 150, 540, 930, 930, 540],
    }
)
print(marker_layout)
plot_marker_layout(marker_layout)

With this, we can now run the ``find_homograpghy()`` method. This method finds the map between the detections and the provided coordinates for each frame. As we did not do detections in every frame, we further provide the skip_frames as used before so that the homographies can be interpolated.

In [ ]:
from pyneon.video import find_homographies

homographies = find_homographies(
    detected_markers,
    marker_layout,
)

In [ ]:
print(homographies.data.head())

In [ ]:
gaze_on_surface = rec.gaze.apply_homographies(homographies)

with this, we can finally transform both gaze and fixation coordinates into the screen's reference frame:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
plt.scatter(
    gaze_on_surface["gaze x [surface coord]"],
    gaze_on_surface["gaze y [surface coord]"],
    s=1,
    alpha=0.5,
)

# plt.scatter(
#     fix_data["fixation x [surface coord]"],
#     fix_data["fixation y [surface coord]"],
#     s=1,
#     c="black",
#     label="Fixations",
# )

plt.plot(
    [360, 1560, 1560, 360, 360],
    [140, 140, 940, 940, 140],
    color="red",
    label="Image outline",
)

plt.xlabel("X Coordinate (surface coord)")
plt.ylabel("Y Coordinate (surface coord)")

plt.xlim(0, 1920)
plt.ylim(0, 1080)

ArUco Markers

In [ ]:
rec = dataset.recordings[1]
detected_markers = rec.scene_video.detect_markers("5x5_100")  